# Temporal-Difference Methods

## TD Prediction: TD(0)

We need to remember again the *prediction problem*:<br>
Given a policy $\pi$, determine $v_\pi$ (or $q_\pi$) from interaction with the environment.

We also need to remember a bit how this problem is solved when using the Monte Carlo approach.<br>
In the Monte Carlo approach the agent interacted with the environment in episodes. After the episode finished, the agent look at every pair state-action in the sequence and if it found a first visit, it calculated the estimated return and used such calculation to update the action value.
Then the agent started again with a new episode and repeated the same procedure for many many many.... many episodes.

This algorithm is guaranteed to be a solution for the prediction problem *<u>as long as we never change the policy between episodes and as long as the algorithm is run long enough</u>*. That is, under these conditions the algorithm guarantees to end up with a nice estimate for the action-value function.

Now we are going to work with the update formula of the Monte Carlo approach, but instead of keeping track of the action-value function, we will keep track of the state values:

$$V(S_t) \longleftarrow V(S_t) + \alpha\big(G_t - V(S_t)\big)$$

The idea behind this equation, is that the value of any state is defined as the expected return that's likely to follow such state if the agent follows the policy.

Remember that the value of a state equals the Expected Return the agent can get when it is in that state and follows a given specific policy?<br>
And remember that this expected return can be expressed by the [Bellman expectation equation](./3%20-%20Solution%20to%20the%20RL%20Framework.ipynb) (you will have to look for the corresponding section).

$
\begin{equation}
\begin{split}
V_\pi(s) & = \mathbb{E}_\pi\big[G_t \mid S_t = s\big] \\
 & = \mathbb{E}_\pi\big[R_{t+1} + \gamma V_\pi(S_{t+1}) \mid S_t = s\big] \\
\end{split}
\end{equation}
$

So, the Bellman expectation equation gives us a way to express the value of any state in terms of the values of the states that could potentially follow (again, when following a given policy).

This equation (the Bellman equation) is used to come up to a different update rule as follows. Instead of calculating an average of the *sampled returns*, it could be calculated **the sampled value of the sum of the immediate reward plus the discounted value of the next state**

$$V(S_t) \longleftarrow V(S_t) + \alpha\big(\boldsymbol{R_{t+1} + \gamma V(S_{t+1})} - V(S_t)\big)$$

With this change in the update rule, the algorithm has now an update step that "understands" the value of a state in terms of the values of the states that follow. With this adaption we have removed any mention of the return that comes at the end of an episode. This means, the algorithm does not need to wait anymore until the episode ends to use its update rule. <u>The algorithm can now use its update rule after every time step</u>.

As an example, let's say the agent is in a state $S_t$ and that it uses the policy to pick its next action $A_t$. After the agent performs such action, it receives a reward and the next state from the environment. Then the prediction algorithm could use this information to calculate the update value function:

![UpdateStepTD](./images/UpdateStepTD.png)

Again; it's important to realize we don't need to wait anymore until the end of the episode to update the values.

<u>This is the first algorithm you can use for the prediction problem when working with continuous tasks.</u>

This update step, updates the state values in terms of the state that follows - we refer to this new estimate as the TD target:

![TDTarget](./images/TDTarget.png)

An explanation of what this equation does: it finds some *middle ground* between the two estimates.<br>
We set the value of $\alpha$ according to which estimate the agent must trust more. To see this more clearly, let's rewrite the update equation as follows:

$$V(S_t) \longleftarrow V(S_t) + \alpha\big(R_{t+1} + \gamma V(S_{t+1}) - V(S_t)\big) $$
$$\downarrow$$
$$V(S_t) \longleftarrow (1 - \alpha)V(S_t) + \alpha\big(R_{t+1} + \gamma V(S_{t+1})\big) $$

The value for $\alpha$ must be a value between $(0,1]$:
 * When $\alpha = 1$, the new estimate is just the TD target.
 * When $\alpha = 0$, the new estimate is just the current estimate.

**Note:** Of course setting $\alpha = 0$ is not desired because then the agent will never really learn.

This means, that to smaller $\alpha$, the less we trust the target when performing an update, and the more we rely on our existing estimate of the state value.
![ValueOfAlpha](./images/ValOfAlpha.png)

So, with this update step formula, we can then write our algorithm. The algorithm's name is *One-Step Temporal Difference* or <b>*TD*</b> for short.<br>
Here, "One-Step" refers to the fact that the value function is updated after any individual step. This algorithm is also refer to as **TD(0)** (TD-Zero).
![TDZeroContinuous](./images/TDZeroContinuous.png)

There is also another version of the algorithm, called **TD(0) Episodic**:

![TDZeroEpisodic](./images/TDZeroEpisodic.png)

## TD Prediction: Action Values

Consider the following sequence of states, actions and rewards:<br>
$S_0, A_0, R_1, S_1$<span style="color:blue">$A_1, R_2, S_2$</span>

For the sequence in black we would have the following update step:<br>
$V(S_0) \longleftarrow V(S_0) + \alpha\big(R_1 + \gamma V(S_1) - V(S_0)\big)$<br>
And for the sequence in blue we would have the following update step:<br>
<span style="color:blue">$V(S_1) \longleftarrow V(S_1) + \alpha\big(R_2 + \gamma V(S_2) - V(S_1)\big)$</span>

In this sequence of states, an agent would take actions following a given policy $\pi$ to receive then a reward and the next state, then it would perform an update step (One-step TD, or TD(0) for continuous tasks) to update the value of the <u>current state</u> (notice the underlying of "current state", this is to denote is really the current state and **not** the next state).

So far so good, but what if we'd like to update the action-value function - therefore returning an estimate of the action values?<br>
The answer to that is that we would need an equation that relates the values of successive state-action pairs. And by now it must be kind of intuitive how such an update equation should look like:

$$Q(S_t, A_t) \longleftarrow Q(S_t, A_t) + \alpha\big( R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)\big)$$

With this equation, the agent will update the values after each action is chosen (instead of doing it after each state received). Same as before, if the agent intereacts with the environment long enough it will be able to reach a pretty good estimate of the action-value function.

## TD Control: Sarsa(0)

Rember the Control Problem?
<center><h4>The Control Problem</h4></center>

$$\text{Determine }\Pi_*$$

<center>(from interaction with the environment)</center>

The algorithm we saw last, estiamtes the action value function following an update rule after each action is chosen. I tried to depict it here as follows:

<span style="color:purple">$S_0,$</span>$A_0, R_1, S_1, $<span style="color:blue">$A_1, $</span>$R_2, S_2, $<span style="color:green">$A_2, $</span>$ ..., R_{t+1}, $<span style="color:red">A_{t+1}</span>

<span style="color:purple">Initialize $Q(s, a) = 0$ for all $s \in S, a \in A(s)$</span><br>
<span style="color:blue">Update $Q(S_0, A_0)$</span><br>
<span style="color:green">Update $Q(S_1, A_1)$</span><br>
<span style="color:red">Update $Q(S_{t+1}, A_{t+1})$</span><br>

It's important to note that the agent uses the <u>same policy</u> at every <u>time step</u> to <u>select the actions</u>.

But how do we know that the polciy cannot be optimized?<br>
In order to produce a control algorithm, we can gradually change the policy so that it becomes more optimal at every time step. For this we can follow an approach that is similar to what is done in the Monte Carlo case; select an action at every time step by using a policy that is Epsilon-Greedy with respect to the current estimate of the action values.<br>

$\pi \longleftarrow \epsilon\text{-greedy}(Q)$:

<span style="color:purple">$S_0,$</span>$A_0, R_1, S_1, $<span style="color:blue">$A_1, $</span>$R_2, S_2, $<span style="color:green">$A_2, $</span>$ ..., R_{t+1}, $<span style="color:red">A_{t+1}</span>

<span style="color:purple">Initialize $Q(s, a) = 0$ for all $s \in S, a \in A(s) \text{ and } \pi \leftarrow \epsilon\text{-greedy}(Q)$</span><br>
<span style="color:blue">Update $Q(S_0, A_0)\text{; }\pi \leftarrow \epsilon\text{-greedy}(Q)$</span><br>
<span style="color:green">Update $Q(S_1, A_1)\text{; }\pi \leftarrow \epsilon\text{-greedy}(Q)$</span><br>
<span style="color:red">Update $Q(S_{t+1}, A_{t+1})\text{; }\pi \leftarrow \epsilon\text{-greedy}(Q)$</span><br>

 1. Set $\epsilon = 1$
 2. $A_0$ and $A_1$ are chosen according to the equal probable random policy.
 3. Then, at all future time steps after an action is chosen, we update the action-value function and construct the corresponding Epsilon-Greedy policy.

"Sarsa(0) is **guaranteed to converge** to the optimal action-value function, as long as the step-size parameter $\alpha$ is sufficiently small, and the **Greedy in the Limit with Infinite Exploration(GLIE)** conditions are met. The GLIE conditions were introduced previously, when we learned about MC control. Although there are many ways to satisfy the GLIE conditions, one method involves gradually decaying the value of $\epsilon$ when construction $\epsilon$-greedy policies.

In particular let $\epsilon_i$ correspond to the i-th time step. Then, if we set $\epsilon_i$ such that:
 * $\epsilon_i >0$ for all time steps i, and
 * $\epsilon_i$ decays to zero in the limit as the time step i approaches infinity (that is $\lim_{i\to\infty}\epsilon_i = 0$),

then the algorithm is guaranteed to yield a good estimate for $q_*$, as long as we run the algorithm for long enough.<br>
A corresponding optimal policy $\pi_*$ can then be quickly obtained by setting $\pi_*(s) = \text{argmax}_{a \in A(s)} q_*(s, a)\text{ for all } s \in S$."

### Sarsa(0) pseudocode

![TDZeroEpisodic](./images/SARSA(0).png)

## TD Control: Sarsamax (aka Q-Learning)

Sarsamax is an algorithm that is build off the Sarsa(0) algorithm and works a little bit different (by the way, this algorithm is also known as Q-Learning).<br>
Everything here works at the begining as usual. The agen starts with some initial policy and initial state values, the agent follows the policy and chooses an action and in consequence it receives a reward and the new state from the environment. But right at this point, after receiving the reward and the new state, it <u>updates the policy before choosing the next action</u>. For this update, instead of using $A_{t+1}$, we'll use $\text{maxarg}Q_{a \in A(s_{t+1}\text{ })}(S_{t+1}, a)$

In the Sarsa case, our update step was one step later and plugged in the action that was selected using the Epsilon Greedy policy. Also, all of the actions that are used (in Sarsa) to update the actions values, are exaclty the actions that the agent experienced during its interaction. This does not have to be the case if we consider using the action from the Greedy Policy, instead of the Epsilon Greedy Policy.

That is exactly what Q-Learning or Sarsamax does. With this in mind, we rewrite the update equation as follows:

$$Q(S_0, A_0) \longleftarrow Q(S_0, A_0) + \alpha\big(R_1 + \gamma\text{maxQ}_{a \in A(S_1)}(S_1, a) - Q(S_0, A_0)\big)$$

In this equation we rely on the fact that the greedy action corresponding to a state is just the one that maximizes the action values for that state.<br>
So, we update the action value for the time step zero using the greedy action policy, then we select the following action(i.e. $A_1$) using the Epsilon Greedy policy corresponding to the action values we just updated.

![Sarsamax_1](./images/Sarsamax_1.png)

Comparing Sarsa(0) and Sarsamax (Q-Learning) we can see that in Sarsa, the update step pushes the action values closer to evaluating whatever Epsilon greedy policy is currently being followed, whereas in Sarsamax the updates attemp directly to approximate the optimal value function at every time step.<br>
Sarsamax guarantees to converge under the same conditions that guarantee the convergence of Sarsa(0).

![Sarsamax_2](./images/Sarsamax_2.png)


### Sarsamax Pseudocode

![Sarsamax_2](./images/SARSAMAX_pseudocode.png)

## TD Control: Expected Sarsa

This algorithm is very similar to Sarsamax with the only difference in Expected Sarsa's update step. Expected Sarsa uses the expected value of the next state action pair and in this expectation it is taken into account the probability of each possible action that the agent can select from the next state:

$$Q(S_t, A_t) \longleftarrow Q(S_t, A_t) + \alpha\Big(R_{t+1} + \gamma\sum_{a \in A}\pi(a \mid S_{t+1})Q(S_{t+1}, a) - Q(S_t, A_t)\Big)$$

Expected Sarsa is guaranteed to converge under the same conditions that guarantee convergence of Sarsa and Sarsamax.
Remember that theoretically, as long as the step-size parameter $\alpha$ is sufficiently small, and the Greedy in the Limit with Infinite Exploration (GLIE) conditions are met, the agent is guaranteed to eventually discover the optimal action-value function (and an associated optimal policy). However, in practice, for all of the algorithms we have discussed, it is common to completely ignore these conditions and still discover an optimal policy. You can see an example of this in the solution notebook.

### Expected Sarsa Pseudocode

![ExpectedSarsaPseudocode](./images/ExpectedSarsa_Pseudocode.png)

By the way, the notebook for this setion is [here](./notesMiniProjects/TemporalDifference/Temporal_Difference.ipynb).

## Comparison

The following image depicts a comparison between the three temporal-difference algorithms discussed on this page:

![Sarsamax_2](./images/SarsaSarsamaxExpectedSarsa_comparison.png)

### Tempora-Difference Methods
Whereas Monte Carlo (MC) prediction methods must wait until the end of an episode to update the value function estimate, temporal-difference (TD) methods update the value function after every time step.

### TD Control
* **Sarsa(0)** (or **Sarsa**) is an on-policy TD control method. It is guaranteed to converge to the optimal action-value function $q_*$​, as long as the step-size parameter $\alpha$ is sufficiently small and $\epsilon$ is chosen to satisfy the Greedy in the Limit with Infinite Exploration (GLIE) conditions.
* **Sarsamax** (or **Q-Learning**) is an off-policy TD control method. It is guaranteed to converge to the optimal action value function $q_*$​, under the same conditions that guarantee convergence of the Sarsa control algorithm.
* **Expected Sarsa** is an on-policy TD control method. It is guaranteed to converge to the optimal action value function $q_∗$​, under the same conditions that guarantee convergence of Sarsa and Sarsamax.

### Analyzing Performance
* On-policy TD control methods (like Expected Sarsa and Sarsa) have better online performance than off-policy TD control methods (like Q-learning).
* Expected Sarsa generally achieves better performance than Sarsa.